## CUSTOMER CHURN MACHINE LEARNING PREDICTION

### Business Understanding
In the highly competitive telecommunications industry, retaining customers is crucial for maintaining revenue and profitability. Customer churn, the phenomenon where customers stop using a company's services, poses a significant challenge. To address this, companies are increasingly leveraging machine learning techniques to predict churn and devise strategies to enhance customer retention.

This project aims to assist a telecommunication company in understanding their customer data, identifying factors contributing to churn, and predicting which customers are likely to churn. By building and evaluating classification models, we can provide actionable insights to help the company mitigate churn and improve customer loyalty.

### Stakeholders
 - Chief Marketing Officer (CMO) 
 - Customer Service Director 
 - Chief Data Officer

### Key Metrics and Success Criteria
 The Model should have an accuracy score of 85% (On balanced data)
 Good models are expected to have an accuracy score of >0.80 or 80% 

### Hypothesis
Null Hypothesis (H0): Customer demographics have no significant effect on churn.
Alternative Hypothesis (H1): Customer demographics have a significant effect on churn. 

### Analytical Questions
1. What is the Churn percentage based on the paymment method?
2. What are the key demographic factors (e.g., gender, age, dependents) that influence customer churn?
3. How frequently do customers who churn interact with customer support compared to those who stay?
4.  How does the tenure of a customer impact their likelihood of churning?
5. Which services (e.g., streaming, international calls, internet service) are most frequently used by customers who churn?
6. How does the type of contract (e.g., month-to-month, one year, two years) impact churn rates?
7. What are the common issues raised by customers who churn during their interactions with customer support?

## Data Understanding